Mounted at /content/gdrive


In [2]:
#Connect to data folder of NerLTR-DTA
root_path = '/data/'
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


#Import libs with versions specified in git repo of NerLTR-DTA

In [3]:
!pip install numpy==1.17.0
!pip install pandas==1.2.3

import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.3 MB 1.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.20.2 requires numpy>=1.18, but you have numpy 1.17.0 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.17.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires numpy>=1.20, but you have numpy 1.17.0 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.17.0 which is incompatible.
pywavelets 1.3.0 requires numpy>=1.17.3, but you have numpy 1.17.0 which is incompatible.
plotnine 0.8.0 requires numpy>=1.19.0, but you have 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 5.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.20.2 requires numpy>=1.18, but you have numpy 1.17.0 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.17.0 which is incompatible.
plotnine 0.8.0 requires numpy>=1.19.0, but you have numpy 1.17.0 which is incompatible.
cmdstanpy 1.0.7 requires numpy>=1.21, but you have numpy 1.17.0 which is incompatible.


#Set paths to the dataset files

In [4]:
path_KIBA_drug_sim = 'KIBA/kiba_drug_sim.txt' #Drug Similarity Matrix SimDD from Kiba Dataset
path_KIBA_target_sim = 'KIBA/kiba_target_sim.txt' #Target Similarity Matrix SimTT from Kiba Dataset
path_KIBA_affinity = 'KIBA/kiba_binding_affinity_v2.txt' #Affinity Matrix AffiDT from Kiba Dataset
save_KIBA_SAF = 'KIBA/SAF'
save_KIBA_AAF = 'KIBA/AAF'

In [5]:
#FOR TESTING ONLY
path_test_drug_sim = 'test/drug_sim.txt' 
path_test_target_sim = 'test/target_sim.txt' 
path_test_affinity = 'test/affi.txt' 
save_test_SAF = 'test/SAF'
save_test_AAF = 'test/AAF'

#Generate SAF

##SSAF

In [6]:
def generate_SSAF_feature(input_array):
  #compute ssaf features for each row of similarity matrix
  ssaf_mean = np.mean(input_array)  #compute mean
  ssaf_median = np.median(input_array)  #compute median
  ssaf_quantile_75 = np.percentile(input_array, 75) #compute 75th quantile
  ssaf_quantile_85 = np.percentile(input_array, 85) #compute 85th quantile
  ssaf_quantile_95 = np.percentile(input_array, 95) #compute 95th quantile
  ssaf_feature = [ssaf_mean, ssaf_median, ssaf_quantile_75, ssaf_quantile_85, ssaf_quantile_95]
  return ssaf_feature

In [7]:
def SSAF_features(input_path:str, output_path: str):
  #load Similarity Matrix
  sim = np.loadtxt(input_path)
  #create dataframe
  ssaf_df = pd.DataFrame(columns = ['mean', 'median', '75q', '85q', '95q'])
  #compute ssaf features
  for i,s in enumerate(sim):
    ssaf_feature = generate_SSAF_feature(s)
    data = {
        'mean':ssaf_feature[0],
        'median': ssaf_feature[1],
        '75q' : ssaf_feature[2],
        '85q' : ssaf_feature[3],
        '95q' : ssaf_feature[4]
    }
    ssaf_df= ssaf_df.append(data, ignore_index=True)
  #save dataframe
  ssaf_df.to_pickle(output_path)

###Drugs

In [ ]:
#Save SSAF features for KIBA dataset drugs
SSAF_features(input_path=path_KIBA_drug_sim, output_path=save_KIBA_SAF+'/SSAF_drugs')

###Proteins

In [ ]:
#Save SSAF features for KIBA dataset targets
SSAF_features(input_path=path_KIBA_target_sim, output_path=save_KIBA_SAF+'/SSAF_targets')

##Test

In [10]:
#Save SSAF features for KIBA dataset drugs
SSAF_features(input_path=path_test_drug_sim, output_path=save_test_SAF+'/SSAF_drugs')
#Save SSAF features for KIBA dataset targets
SSAF_features(input_path=path_test_target_sim, output_path=save_test_SAF+'/SSAF_targets')

##ASAF

In [11]:
def compute_mode(sample):

  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]] #returns a list of all modes

def generate_ASAF_feature(input_array):
  #compute asaf features for each row of affinity matrix
  asaf_mean1 = np.mean(np.nan_to_num(input_array))   # compute mean including nan (full length of affinity e.g. KIBA=229)

  #removing all nan from array
  no_nan_array = input_array[~np.isnan(input_array)]
  asaf_mean2 = np.mean(no_nan_array)  # compute mean excluding nan (all nan values ignored)
  asaf_number = len(no_nan_array)  #number of non-nan values
  asaf_mode = np.mean(compute_mode(no_nan_array))  #compute mean of all modes
  asaf_quantile_25 = np.percentile(no_nan_array, 25) #compute 25th quantile
  asaf_quantile_50 = np.percentile(no_nan_array, 50)  #compute 50th quantile
  asaf_quantile_75 = np.percentile(no_nan_array, 75)  #compute 75th quantile

  #Sort arary for lowest and highest values
  sorted_index_array = np.argsort(no_nan_array)# sorted array
  sorted_array = no_nan_array[sorted_index_array]  # sorted array from lowest to highest
  n=5 #n highest/lowest values
  asaf_5_high = sorted_array[-n:]
  asaf_5_low = sorted_array[:n]

  asaf_feature = [asaf_mean1, asaf_mean2, asaf_number, asaf_mode, asaf_quantile_25, asaf_quantile_50, asaf_quantile_75, asaf_5_high, asaf_5_low]
  return asaf_feature

In [12]:
def ASAF_features(input_path:str, output_path: str, drugs=False):
  #load Affinity Matrix
  aff = np.loadtxt(input_path)
  #if targets --> transpose Affinity Matrix
  if drugs==False:
    aff = aff.T
  #create dataframe
  asaf_df = pd.DataFrame(columns = ['mean_1', 'mean_2', 'number', 'mode', '25q', '50q', '75q', '5_high', '5_low'])
  #compute asaf features
  for i,a in enumerate(aff):
    asaf_feature = generate_ASAF_feature(a)
    data = {
        'mean_1':asaf_feature[0],
        'mean_2':asaf_feature[1],
        'number':asaf_feature[2],
        'mode': asaf_feature[3],
        '25q' : asaf_feature[4],
        '50q' : asaf_feature[5],
        '75q' : asaf_feature[6],
        '5_high' : asaf_feature[7],
        '5_low' : asaf_feature[8],
    }
    asaf_df= asaf_df.append(data, ignore_index=True)
  #save dataframe
  asaf_df.to_pickle(output_path)

###Drugs

In [ ]:
#Save SSAF features for KIBA dataset drugs
ASAF_features(input_path=path_KIBA_affinity, output_path=save_KIBA_SAF+'/ASAF_drugs', drugs=True)

###Targets

In [ ]:
#Save SSAF features for KIBA dataset targets
ASAF_features(input_path=path_KIBA_affinity, output_path=save_KIBA_SAF+'/ASAF_targets', drugs=False)

##Test

In [17]:
#Save SSAF features for KIBA dataset drugs
ASAF_features(input_path=path_test_affinity, output_path=save_test_SAF+'/ASAF_drugs', drugs=True)
#Save SSAF features for KIBA dataset targets
ASAF_features(input_path=path_test_affinity, output_path=save_test_SAF+'/ASAF_targets', drugs=False)

#Generate AAF

##Compute Share matrix

In [18]:
def generate_share_matrix_KIBA(input_path:str, output_path:str):
  #load Affinity Matrix
  aff = np.loadtxt(input_path)
  aff[aff > -1.0] = 1 #set all values that are not nan to 1 for easier comparison
  share_matrix_drugs = np.zeros((len(aff),len(aff)))
  for i,row in enumerate(aff): #loop through each row in aff matrix for drug-drug share --> initial row vector = row 
    for j, compare_row in enumerate(aff): #loop through each row and compare with initial row vector
      share_matrix_drugs[i][j]=np.count_nonzero(row==compare_row) #count number of places where both arrays are 1
  np.save(output_path+'share_drugs', share_matrix_drugs)

  #do this again with the transposed matrix to get share  matrix for targets
  aff_transposed = aff.T
  share_matrix_targets = np.zeros((len(aff_transposed),len(aff_transposed)))
  for i,row in enumerate(aff_transposed): #loop through each row in aff matrix for drug-drug share --> initial row vector = row 
    for j, compare_row in enumerate(aff_transposed): #loop through each row and compare with initial row vector
      share_matrix_targets[i][j]=np.count_nonzero(row==compare_row)
  np.save(output_path+'share_targets', share_matrix_targets)


###Drugs and Targets

In [ ]:
#Save drug and target share matrix for KIBA dataset
generate_share_matrix_KIBA(input_path=path_KIBA_affinity, output_path='KIBA/')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


##Test

In [20]:
#Save SSAF features for KIBA dataset drugs
generate_share_matrix_KIBA(input_path=path_test_affinity, output_path='test/')

##Index for GAAF

###Index with Threshold 1 from similarity matrix

In [42]:
def compute_index_threshold_1(input_path_similarity:str, input_path_ssaf:str, output_path:str):
  #load the similarity matrix
  sim_matrix=np.loadtxt(input_path_similarity)
  #load ssaf features that are used as thresholds
  with open(input_path_ssaf, "rb") as fp:   # Unpickling SSAF features
    ssaf_feature = pickle.load(fp)
  #loop through each row of the similarity matrix
  gaaf_t1_index=[]
  for i,row in enumerate(sim_matrix):
    threshold_mean = ssaf_feature['mean'][i]
    threshold_median = ssaf_feature['median'][i]
    threshold_75 = ssaf_feature['75q'][i]
    threshold_85 = ssaf_feature['85q'][i]
    threshold_95 = ssaf_feature['95q'][i]
    #get index that have value higher than the thresholds
    index_mean = [idx for idx, val in enumerate(row) if val > threshold_mean]
    index_median = [idx for idx, val in enumerate(row) if val > threshold_median] 
    index_75 = [idx for idx, val in enumerate(row) if val > threshold_75] 
    index_85 = [idx for idx, val in enumerate(row) if val > threshold_85]
    index_95 = [idx for idx, val in enumerate(row) if val > threshold_95]
    #combine the index to one list 
    gaaf_t1_index.append(index_mean+index_median+index_75+index_85+index_95)
  #save to pickl file
  with open(output_path, "wb") as fp:  
    pickle.dump(gaaf_t1_index, fp)
  return gaaf_t1_index

###Drugs

In [ ]:
#Save GAAF index for KIBA dataset
gaaf_index_t1_KIBA_drugs = compute_index_threshold_1(input_path_similarity=path_KIBA_drug_sim, input_path_ssaf=save_KIBA_SAF+'/SSAF_drugs', output_path=save_KIBA_AAF+'/GAAF_T1_index_drugs')

###Targets

In [ ]:
#Save GAAF index for KIBA dataset
gaaf_index_t1_KIBA_drugs = compute_index_threshold_1(input_path_similarity=path_KIBA_target_sim, input_path_ssaf=save_KIBA_SAF+'/SSAF_targets', output_path=save_KIBA_AAF+'/GAAF_T1_index_targets')

##Test

In [43]:
#Save GAAF index for test dataset
gaaf_index_t1_test_drugs = compute_index_threshold_1(input_path_similarity=path_test_drug_sim, input_path_ssaf=save_test_SAF+'/SSAF_drugs', output_path=save_test_AAF+'/GAAF_T1_index_drugs')
#Save GAAF index for test dataset
gaaf_index_t1_test_targets = compute_index_threshold_1(input_path_similarity=path_test_target_sim, input_path_ssaf=save_test_SAF+'/SSAF_targets', output_path=save_test_AAF+'/GAAF_T1_index_targets')

In [44]:
gaaf_index_t1_test_drugs

[[0, 2, 0, 2, 0, 0, 0],
 [1, 2, 1, 2, 1, 1, 1],
 [0, 1, 2, 1, 2, 2, 2, 2],
 [3, 4, 3, 4, 3, 3, 3],
 [3, 4, 3, 4, 4, 4, 4]]

###Index with Threshold 2 from share matrix

In [23]:
def compute_index_threshold_2(input_path:str, output_path:str):
  #load the share matrix
  share_matrix=np.load(input_path)
  #loop through each row of the share matrix
  gaaf_t2_index=[]
  for row in share_matrix:
    threshold_75=np.percentile(row, 75)  #compute 75th quantile
    threshold_85=np.percentile(row, 85)  #compute 85th quantile
    threshold_95=np.percentile(row, 95)  #compute 95th quantile
    #get index that have value higher than the thresholds
    index_75 = [idx for idx, val in enumerate(row) if val > threshold_75] 
    index_85 = [idx for idx, val in enumerate(row) if val > threshold_85]
    index_95 = [idx for idx, val in enumerate(row) if val > threshold_95]
    #combine the index to one list 
    gaaf_t2_index.append(index_75+index_85+index_95)
  #save to pickl file
  with open(output_path, "wb") as fp:  
    pickle.dump(gaaf_t2_index, fp)
  return gaaf_t2_index

###Drugs

In [ ]:
#Save GAAF index for KIBA dataset drugs
gaaf_index_t2_KIBA_drugs = compute_index_threshold_2(input_path='KIBA/share_drugs.npy', output_path=save_KIBA_AAF+'/GAAF_T2_index_drugs')

###Targets

In [ ]:
#Save GAAF index for KIBA dataset drugs
gaaf_index_t2_KIBA_drugs = compute_index_threshold_2(input_path='KIBA/share_targets.npy', output_path=save_KIBA_AAF+'/GAAF_T2_index_targets')

##Test

In [34]:
#Save GAAF index for test dataset drugs
gaaf_index_t2_test_drugs = compute_index_threshold_2(input_path='test/share_drugs.npy', output_path=save_test_AAF+'/GAAF_T2_index_drugs')
#Save GAAF index for test dataset drugs
gaaf_index_t2_test_targets = compute_index_threshold_2(input_path='test/share_targets.npy', output_path=save_test_AAF+'/GAAF_T2_index_targets')

In [35]:
gaaf_index_t2_test_drugs

[[], [], [2, 2, 2], [3, 3, 3], []]

##Generate GAAF feature

In [67]:
#Concat to one long vector and take min, max, upepr quartile , median, lower quartile, mean and mode
def generate_GAAF_feature(input_path_t1_index:str, input_path_t2_index:str, input_path_features:str, output_path: str):
  with open(input_path_t1_index, "rb") as fp:   # Unpickling GAAF_index index from Threshold 1
    index_t1_file = pickle.load(fp)

  with open(input_path_t2_index, "rb") as fp:   # Unpickling GAAF_index index from Threshold 2
    index_t2_file = pickle.load(fp)

  with open(input_path_features, "rb") as fp:   # Unpickling ASAF features index
    feature_file = pickle.load(fp)
  #create dataframe
  gaaf_df = pd.DataFrame(columns = ['min', 'max', 'upper_q', 'median', 'lower_q', 'mean', 'mode'])
  for i,row in enumerate(index_t1_file): #loop through each row of index file
    combined_index = row+index_t2_file[i] #combine index of thresold 1 and thresold 2
    asaf_feature = [] # initial vector used to cumpute the gaaf features
    for idx in combined_index: #loop through each index in the row to extrax the according ASAF features 
      asaf_feature.append(feature_file['mean_1'][idx])
      asaf_feature.append(feature_file['mean_2'][idx])
      asaf_feature.append(feature_file['number'][idx])
      asaf_feature.append(feature_file['mode'][idx])
      asaf_feature.append(feature_file['25q'][idx])
      asaf_feature.append(feature_file['50q'][idx])
      asaf_feature.append(feature_file['75q'][idx])
      #Append the 5 high/low values not as an array but as singl values
      for high in feature_file['5_high'][idx]:
        asaf_feature.append(high)
      for low in feature_file['5_low'][idx]:
        asaf_feature.append(low)
    #now compute the gaaf features from asaf features
    data = {
        'min':np.min(asaf_feature),
        'max':np.max(asaf_feature),
        'upper_q':np.percentile(asaf_feature, 75),
        'median': np.percentile(asaf_feature, 50),
        'lower_q' : np.percentile(asaf_feature, 25),
        'mean' : np.mean(asaf_feature),
        'mode' : np.mean(compute_mode(asaf_feature))
    }
    gaaf_df= gaaf_df.append(data, ignore_index=True)
  with open(output_path, "wb") as fp:  
    pickle.dump(gaaf_df, fp)

###Drugs

In [ ]:
#Save GAAF features for KIBA dataset
generate_GAAF_feature(input_path_t1_index=save_KIBA_AAF+'/GAAF_T1_index_drugs', 
                      input_path_t2_index=save_KIBA_AAF+'/GAAF_T2_index_drugs', 
                      input_path_features=save_KIBA_SAF+'/ASAF_drugs', 
                      output_path=save_KIBA_AAF+'/GAAF_drugs')

###Targets

In [ ]:
#Save GAAF features for KIBA dataset
generate_GAAF_feature(input_path_t1_index=save_KIBA_AAF+'/GAAF_T1_index_targets', 
                      input_path_t2_index=save_KIBA_AAF+'/GAAF_T2_index_targets', 
                      input_path_features=save_KIBA_SAF+'/ASAF_targets', 
                      output_path=save_KIBA_AAF+'/GAAF_targets')

##Test

In [68]:
#Save GAAF features for KIBA dataset
generate_GAAF_feature(input_path_t1_index=save_test_AAF+'/GAAF_T1_index_drugs', 
                      input_path_t2_index=save_test_AAF+'/GAAF_T2_index_drugs', 
                      input_path_features=save_test_SAF+'/ASAF_drugs', 
                      output_path=save_test_AAF+'/GAAF_drugs')
#Save GAAF features for KIBA dataset
generate_GAAF_feature(input_path_t1_index=save_test_AAF+'/GAAF_T1_index_targets', 
                      input_path_t2_index=save_test_AAF+'/GAAF_T2_index_targets', 
                      input_path_features=save_test_SAF+'/ASAF_targets', 
                      output_path=save_test_AAF+'/GAAF_targets')

##Index for AAAF

In [6]:
def AAAF_indexes(input_path:str, output_path: str):
  #load similary matrix
  sim = np.loadtxt(input_path)
  #Sort arary for highes and lowest values
  top_5_similar_index=[]
  n=5 #n most similar index
  for s in sim:
    top_5_similar_index.append(np.argsort(-s)[1:n+1].tolist()) #sort and get 5 most similar
  #get 5 similar for each of the 5 most similar 
  aaaf_index=[]
  for row in top_5_similar_index:
    all_25_index=[]
    all_25_index.append([top_5_similar_index[i] for i in row])
    flat_list = [subitem for item in all_25_index for subitem in item]
    flat_list = [subitem for item in flat_list for subitem in item]
    aaaf_index.append(flat_list)

  #save to pickl file
  with open(output_path, "wb") as fp:  
    pickle.dump(aaaf_index, fp)
  return aaaf_index

###Drugs

In [7]:
#Save AAAF index for KIBA dataset
aaaf_index_KIBA_drugs = AAAF_indexes(input_path=path_KIBA_drug_sim, output_path=save_KIBA_AAF+'/AAAF_index_drugs')

###Targets

In [9]:
#Save AAAF index for KIBA dataset
aaaf_index_KIBA_drugs = AAAF_indexes(input_path=path_KIBA_target_sim, output_path=save_KIBA_AAF+'/AAAF_index_targets')

##Test

In [111]:
#Save AAAF index for test dataset
aaaf_index_test_drugs = AAAF_indexes(input_path=path_test_drug_sim, output_path=save_test_AAF+'/AAAF_index_drugs')
#Save AAAF index for test dataset
aaaf_index_test_targets = AAAF_indexes(input_path=path_test_target_sim, output_path=save_test_AAF+'/AAAF_index_targets')

##Gernerate AAAF feature 

In [11]:
def generate_AAAF_feature(input_path_index:str, input_path_features:str, output_path: str):
  with open(input_path_index, "rb") as fp:   # Unpickling AAAF_index index
    index_file = pickle.load(fp)

  with open(input_path_features, "rb") as fp:   # Unpickling ASAF features index
    feature_file = pickle.load(fp)
  
  aaaf_features=[]
  for row in index_file: #loop through each row of index file --> contains 25 index
    asaf_features = []
    for idx in row: #loop through each index in the row to extrax the according ASAF features 
      asaf_features.append(feature_file['mean_1'][idx])
      asaf_features.append(feature_file['mean_2'][idx])
      asaf_features.append(feature_file['number'][idx])
      asaf_features.append(feature_file['mode'][idx])
      asaf_features.append(feature_file['25q'][idx])
      asaf_features.append(feature_file['50q'][idx])
      asaf_features.append(feature_file['75q'][idx])
      #Append the 5 high/low values not as an array but as singl values
      for high in feature_file['5_high'][idx]:
        asaf_features.append(high)
      for low in feature_file['5_low'][idx]:
        asaf_features.append(high)
    aaaf_features.append(asaf_features)
  #save to pickl file
  with open(output_path, "wb") as fp:  
    pickle.dump(aaaf_features, fp)

###Drugs

In [12]:
#Save AAAF features for KIBA dataset
generate_AAAF_feature(input_path_index=save_KIBA_AAF+'/AAAF_index_drugs', input_path_features=save_KIBA_SAF+'/ASAF_drugs', output_path=save_KIBA_AAF+'/AAAF_drugs')

###Targets

In [14]:
#Save AAAF features for KIBA dataset
generate_AAAF_feature(input_path_index=save_KIBA_AAF+'/AAAF_index_targets', input_path_features=save_KIBA_SAF+'/ASAF_targets', output_path=save_KIBA_AAF+'/AAAF_targets')

##Test

In [94]:
#Save AAAF features for test dataset
generate_AAAF_feature(input_path_index=save_test_AAF+'/AAAF_index_drugs', input_path_features=save_test_SAF+'/ASAF_drugs', output_path=save_test_AAF+'/AAAF_drugs')
#Save AAAF features for KIBA dataset
generate_AAAF_feature(input_path_index=save_test_AAF+'/AAAF_index_targets', input_path_features=save_test_SAF+'/ASAF_targets', output_path=save_test_AAF+'/AAAF_targets')